In [1]:
import numpy as np
import pickle
import gzip
from PIL import Image
from tqdm import tqdm_notebook
import os
from matplotlib import pyplot as plt
from statistics import mode

import keras
from keras.datasets import mnist
from keras.layers import Dense
from keras.models import Sequential

from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.


## Load MNIST

In [2]:
filename = 'mnist.pkl.gz'
f = gzip.open(filename, 'rb')
training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
f.close()

## Loading training, validation and testing data for MNIST

In [3]:
Mtraining_data = np.array(training_data[0]);
Mtraining_target = np.array(training_data[1]);
Mvalidation_data = np.array(validation_data[0]);
Mvalidation_target = np.array(validation_data[1]);
Mtest_data = np.array(test_data[0]);
Mtest_target = np.array(test_data[1]);
#Mtraining_data = np.concatenate((Mtraining_data,Mvalidation_data),axis=0);
#Mtraining_target = np.concatenate((Mtraining_target,Mvalidation_target),axis=0);
print("Shapes of MNIST training data: ");
print(np.shape(Mtraining_data));
print(np.shape(Mtraining_target));
#print(np.shape(Mtraining_data))
print("Shapes of MNIST testing data: ");
print(np.shape(Mtest_data));
print(np.shape(Mtest_target));

Shapes of MNIST training data: 
(50000, 784)
(50000,)
Shapes of MNIST testing data: 
(10000, 784)
(10000,)


## Load USPS 

In [5]:
USPSMat  = []
USPSTar  = []
curPath  = 'USPSdata/Numerals'
savedImg = []

for j in range(0,10):
    curFolderPath = curPath + '/' + str(j)
    imgs =  os.listdir(curFolderPath)
    for img in imgs:
        curImg = curFolderPath + '/' + img
        if curImg[-3:] == 'png':
            img = Image.open(curImg,'r')
            img = img.resize((28, 28))
            savedImg = img
            imgdata = (255-np.array(img.getdata()))/255
            USPSMat.append(imgdata)
            USPSTar.append(j)

## Loading training, validation and testing data for USPS

In [6]:
print("Shapes of USPS testing data: ");
print(np.shape(USPSMat))
print(np.shape(USPSTar))
USPSMat = np.array(USPSMat);
USPSTar = np.array(USPSTar);

Shapes of USPS testing data: 
(19999, 784)
(19999,)


## Functions for Multiclass Logistic Regression Using Softmax

In [7]:
def get_activation(X,W):
    return np.dot(X,W);

def get_model(A):
    exp_A = np.exp(A);
    for i in exp_A:
        sum_exp = np.sum(i); 
        for j in range(len(i)):
            i[j] = i[j]/sum_exp;
    return exp_A;

def get_hot_target(t,y):
    i=0;
    for j in t:
        a = y[i]
        j[a] = j[a] + 1;
        i = i+1
    return t

def get_cross_entropy(X,W,Y):
    a = np.dot(X,W);
    exp_a = np.exp(a);
    sum_ex = 0;
    for i in exp_a:
        sum_ex = np.sum(i);
        for j in range(len(i)):
            i[j] = i[j]/sum_ex;
    k=0;
    count = 0;
    for i in exp_a:
        j = np.argmax(i);
        if(j == Y[k]):
            count = count+1;
        k = k+1;    
    Loss = 0
    ln_y = -np.log(exp_a);
    i=0;
    for j in ln_y:
        x = Y[i];
        Loss = Loss + j[x];
        i=i+1;
    return (1/len(X))*Loss,float((count*100)/len(X));

## Gradient Descent for Softmax Regression

In [ ]:
Weights = np.random.rand(784,10);
t_values = np.zeros((50000,10));
La = 2;
learning_rate = 0.000012;
t_values = get_hot_target(t_values,Mtraining_target);
LTr = [];
ATr = [];
LV = [];
AV = [];
LT = [];
AT = [];
LU = [];
AU = [];
ite = [];
for i in tqdm_notebook(range(0,100)):
    Activation = get_activation(Mtraining_data,Weights);
    Model = get_model(Activation);
    Delta_EW = np.matmul(np.transpose(Model - t_values),Mtraining_data);
    La_Delta_EW  = np.dot(La,Weights);
    Delta_E = np.add(Delta_EW, np.transpose(La_Delta_EW));
    Delta_W = -np.dot(learning_rate,Delta_E);
    W_next = Weights + np.transpose(Delta_W);
    Weights = W_next;
    #LossTr,accuracyTr = get_cross_entropy(Mtraining_data,Weights,Mtraining_target);
    #LossV,accuracyV = get_cross_entropy(Mvalidation_data,Weights,Mvalidation_target);
    LossT,accuracyT = get_cross_entropy(Mtest_data,Weights,Mtest_target);
    LossU,accuracyU = get_cross_entropy(USPSMat,Weights,USPSTar);
    #ATr.append(accuracyTr);
    #LTr.append(LossTr);
    #AV.append(accuracyV);
    #LV.append(LossV);
    AT.append(accuracyT);
    LT.append(LossT);
    LU.append(LossU);
    AU.append(accuracyU);
    ite.append(i);


In [ ]:
plt.plot(ite, LT) 
plt.xlabel('Iterations') 
plt.ylabel('cross entropy testing Values')   
plt.title('cross entropy testing vs Iterations') 
plt.show()

plt.plot(ite, AT) 
plt.xlabel('Iterations') 
plt.ylabel('Testing accuracy Values')   
plt.title('Testing accuracy vs Iterations') 
plt.show()
#print(A[999]);
plt.plot(ite, LU) 
plt.xlabel('Iterations') 
plt.ylabel('cross entropy testing Values for USPS')   
plt.title('cross entropy testing vs Iterations') 
plt.show()

plt.plot(ite, AU) 
plt.xlabel('Iterations') 
plt.ylabel('Testing accuracy Values for USPS')   
plt.title('Testing accuracy vs Iterations') 
plt.show()

In [77]:
print("Accuracy for MNIST test data: ");
print(max(AT));
print("Accuracy for USPS dataset: ");
print(max(AU));

Accuracy for MNIST test data: 
87.34
Accuracy for USPS dataset: 
27.2063603180159


In [78]:
Mtraining_data = np.concatenate((Mtraining_data,Mvalidation_data),axis=0);
Mtraining_target = np.concatenate((Mtraining_target,Mvalidation_target),axis=0);
print("Shapes of MNIST training data: ");
print(np.shape(Mtraining_data));
print(np.shape(Mtraining_target));
#print(np.shape(Mtraining_data))
print("Shapes of MNIST testing data: ");
print(np.shape(Mtest_data));
print(np.shape(Mtest_target));

Shapes of MNIST training data: 
(70000, 784)
(70000,)
Shapes of MNIST testing data: 
(10000, 784)
(10000,)


## Keras DNN Implementation 

In [79]:
#(x_train, y_train),(x_test, y_test) = mnist.load_data()
num_classes=10
image_vector_size=28*28
#x_train = x_train.reshape(x_train.shape[0], image_vector_size)
#x_test = x_test.reshape(x_test.shape[0], image_vector_size)
x_train = Mtraining_data;
u_x_test = USPSMat;
y_train = keras.utils.to_categorical(Mtraining_target, num_classes);
u_y_test = keras.utils.to_categorical(USPSTar, num_classes);
x_test = Mtest_data;
y_test = keras.utils.to_categorical(Mtest_target, num_classes);
#print(np.shape(x_test))
#print(np.shape(y_test))
#print(np.shape(x_train))
#print(np.shape(y_train))
#print(y_train[0])
image_size = 784
model = Sequential()
model.add(Dense(units=32, activation='sigmoid', input_shape=(image_size,)))
model.add(Dense(units=num_classes,activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=128, epochs=100, verbose=True,validation_split=.1)
print("Loss and accuracy for MNIST Test data: ");
loss,accuracy = model.evaluate(x_test, y_test, verbose = True);
print(loss)
print(accuracy)
print("Loss and accuracy for USPS data: ");
loss,accuracy = model.evaluate(u_x_test, u_y_test, verbose = True);
print(loss)
print(accuracy)

Train on 63000 samples, validate on 7000 samples
Epoch 1/100
63000/63000 [==============================] - 1s 17us/step - loss: 2.1125 - acc: 0.4226 - val_loss: 1.8869 - val_acc: 0.6357
Epoch 2/100
63000/63000 [==============================] - 1s 11us/step - loss: 1.7247 - acc: 0.6581 - val_loss: 1.5339 - val_acc: 0.7251
Epoch 3/100
63000/63000 [==============================] - 1s 11us/step - loss: 1.4198 - acc: 0.7276 - val_loss: 1.2552 - val_acc: 0.7957
Epoch 4/100
63000/63000 [==============================] - 1s 11us/step - loss: 1.1876 - acc: 0.7774 - val_loss: 1.0495 - val_acc: 0.8337
Epoch 5/100
63000/63000 [==============================] - 1s 13us/step - loss: 1.0183 - acc: 0.8090 - val_loss: 0.9008 - val_acc: 0.8519
Epoch 6/100
63000/63000 [==============================] - 1s 13us/step - loss: 0.8953 - acc: 0.8256 - val_loss: 0.7921 - val_acc: 0.8629
Epoch 7/100
63000/63000 [==============================] - 1s 13us/step - loss: 0.8039 - acc: 0.8384 - val_loss: 0.7100 - v

## SVM Implementation

In [80]:
svm_x_train = Mtraining_data;
svm_y_train = Mtraining_target;
svm_x_test = Mtest_data;
svm_y_test = Mtest_target;
svm_u_x_test = USPSMat;
svm_u_y_test = USPSTar;
#print(np.shape(x_train));
#print(np.shape(y_train));
#print(np.shape(x_test));
#print(np.shape(y_test));
classifier1 = SVC(kernel='rbf', C=2, gamma = 0.05, verbose =1);
classifier1.fit(svm_x_train, svm_y_train)
accuracy = classifier1.score(svm_x_test,svm_y_test);
predict = classifier1.predict(svm_x_test);
print("Accuracy for SVM on MNIST Data :")
print(accuracy);

[LibSVM]Accuracy for SVM on MNIST Data :
0.9836


In [82]:
accuracy1 = classifier1.score(svm_u_x_test,svm_u_y_test);
print("Accuracy for USPS dataset")
print(accuracy1);

Accuracy for USPS dataset
0.27391369568478424


## Random forest implementation

In [84]:
rf_x_train = Mtraining_data;
rf_y_train = Mtraining_target;
rf_x_test = Mtest_data;
rf_y_test = Mtest_target;
rf_u_x_test = USPSMat;
rf_u_y_test = USPSTar;
classifier2 = RandomForestClassifier(n_estimators=100, verbose=1);
classifier2.fit(rf_x_train, rf_y_train)
print("Accuracy for MNIST dataset: ");
accuracy = classifier2.score(rf_x_test,rf_y_test);
print(accuracy);
print("Accuract for USPS dataset: ");
accuracy = classifier2.score(rf_u_x_test,rf_u_y_test);
print(accuracy);

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   40.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy for MNIST dataset: 


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.9712
Accuract for USPS dataset: 
0.39056952847642384


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished


## Confusion Matrices and Majority Voting Implementation :

In [67]:
f1 = open('softmax_predictions','rb');
smp = pickle.load(f1,encoding='latin1');
f1.close();
#print(smp);
#print(smp);
smp = np.array(smp);

In [20]:
f2 = open('dnn_predictions','rb');
dnnp = pickle.load(f2,encoding = 'latin1');
f2.close();
dnnp = np.array(dnnp)

array([7, 2, 1, ..., 4, 5, 6])

In [21]:
f3 = open('SVM_predictions','rb');
svmp = pickle.load(f3, encoding='latin1');
f3.close();
#print(svmp);
svmp

array([7, 2, 1, ..., 4, 5, 6])

In [22]:
f4 = open('random_forest_predictions','rb');
rfp = pickle.load(f4,encoding="latin1");
f4.close();
rfp

array([7, 2, 1, ..., 4, 5, 6])

## Confusion Matrix for Softmax regression :

In [23]:
cf1 = confusion_matrix(Mtest_target, smp)
print(cf1);

[[ 935    0    5    5    0   15   11    1    7    1]
 [   0 1095    4    6    0    5    3    0   22    0]
 [  18    8  865   20   19    0   17   23   51   11]
 [   4    4   21  878    0   47    5   16   21   14]
 [   1    2    5    2  864    5   26    1    9   67]
 [  18    6    8   62   13  682   19    7   61   16]
 [  15    6   13    2   16   16  879    1   10    0]
 [   2   18   38    4    9    1    0  910    3   43]
 [   5   16   10   46   19   42   13   15  788   20]
 [   9    7    5    8   63   20    1   28   14  854]]


## Confusion Matrix for DNN :

In [24]:
cf2 = confusion_matrix(Mtest_target, dnnp)
print(cf2);

[[ 959    0    2    2    0    5    8    1    3    0]
 [   0 1112    3    2    0    1    4    2   11    0]
 [   9    4  937   14   17    0   13   11   25    2]
 [   3    1   16  926    0   24    2   17   15    6]
 [   1    1    5    0  921    2    9    2    6   35]
 [   9    3    2   41    5  784   14    4   21    9]
 [  13    3    5    0   10   11  913    1    2    0]
 [   3   10   25    4    6    0    0  950    2   28]
 [   8    9    5   16    9   22   14    9  877    5]
 [  10    7    2   11   38   10    0   12    6  913]]


## Confusion Matrix for SVM :

In [25]:
cf3 = confusion_matrix(Mtest_target, svmp)
print(cf3);

[[ 973    0    1    0    0    2    1    1    2    0]
 [   0 1127    3    1    0    1    1    1    1    0]
 [   4    0 1015    0    1    0    0    6    6    0]
 [   0    0    2  995    0    3    0    6    4    0]
 [   0    0    3    0  966    0    4    0    2    7]
 [   2    0    0    5    1  878    2    1    2    1]
 [   4    2    0    0    2    3  946    0    1    0]
 [   0    3   10    1    1    0    0 1004    2    7]
 [   1    0    1    3    1    2    0    2  961    3]
 [   3    3    2    6   10    2    0    6    6  971]]


## Confusion Matrix for Random Forest :

In [27]:
cf4 = confusion_matrix(Mtest_target, rfp)
print(cf4);

[[ 969    0    0    0    0    4    3    1    2    1]
 [   0 1122    3    3    0    1    3    1    2    0]
 [   6    0  999    4    2    0    4    9    8    0]
 [   1    0   10  972    0    9    0    9    7    2]
 [   1    0    1    0  955    0    5    0    3   17]
 [   2    1    0   12    2  860    5    2    5    3]
 [   5    3    0    0    2    3  942    0    3    0]
 [   2    2   17    1    0    1    0  993    3    9]
 [   5    0    3   11    5    3    2    4  933    8]
 [   5    4    1   11    9    3    1    4   10  961]]


## Majority Voting :

In [57]:
L = [];
E = [];
for i in range(0,10000):
    L.append(smp[i]);
    L.append(dnnp[i]);
    L.append(svmp[i]);
    L.append(rfp[i]);
    #print(mode(L));
    #E.append(max(L, key = L.count))
    try:
        E.append(mode(L));
    except:
        #print("error in  index: "+str(i));
        E.append(svmp[i]);
        continue
    L=[];
print(np.shape(E));
#print(E[9698]);

(10000,)


In [66]:
Eaccuracy = 0;
count = 0
k=0;
for i in range(0,10000):
    if(Mtest_target[i] == E[i]):
        count = count+1;
print(count);
Eaccuracy = float((count*100)/len(E));
print("Accuracy for Majority voting");
print(Eaccuracy);

9738
97.38
